In [1]:
#importing Packages
import tweepy
import time
from kafka import KafkaConsumer, KafkaProducer
import json

In [2]:
# Twitter Keys
consumer_key = "hBS2M9407C5vDTXodUMtfOTh9"
consumer_secret = "rh9UuGlqNhzHm6GYo0koWbJeaFVO0Ml72uBEcGd9rYS6ZSRIbO"
access_token = "1342336999-AjXWWz4qadipKXLwPxd6MyZsE8ZQyQvxgRJnf7H"
access_token_secret = "7xiPV4MjeqwINKsWd197bZ3DWeigc6Ez0hfoRP6jMhiTB"

# Creating the authentication object
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

# Creating the API object by passing in auth information
api = tweepy.API(auth)

#Connection Checking
try:
    api.verify_credentials()
    print("Authentication OK")
except:
    print("Error during authentication")

Authentication OK


In [3]:
#Creating Kafka-Prodcuer for Procducing data from Twitter.
producer = KafkaProducer(bootstrap_servers='192.168.99.100:9092')
topic_name = 'tweet'

In [4]:
# Search words and filters
words = "iphoneXR"
date_since = "2019-08-08"
new_search = words + " -filter:retweets"

In [5]:
def get_twitter_data():
    #Searching keywords using tweepy
    tweets = tweepy.Cursor(api.search, q=new_search, lang="en", since=date_since).items(3)
    for tweet in tweets:
        record = ''
        record += str(tweet.user.id_str) #userId
        record += ',' 
        record += str(tweet.user.screen_name) #userscreen name
        record += ','
        record += str(tweet.user.followers_count) #followers count
        record += ','
        record += str(tweet.user.location) #location
        record += ','
        record += str(tweet.text) #tweet
        producer.send (topic_name,json.dumps(record).encode('utf-8'))

In [7]:
get_twitter_data() # for single run

In [8]:
#For Multiple Run
def periodic_work(interval):
    while True:
        get_twitter_data()
        #interval should be an integer, the number of seconds to wait
        time.sleep(interval)
periodic_work(30 * 0.1)  # get data every couple of minutes

KeyboardInterrupt: 